In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
dataframe = pd.read_csv("/content/drive/My Drive/ML Project Taxi Fair/Outlier_Rmoved_with_drop_nan_train.csv")

In [ ]:
dataframe.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,1
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,1
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,1
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,1
4,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,1


In [ ]:
dataframe.isna().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
label                        0
dtype: int64

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


In [ ]:
X = dataframe.drop(columns=['label']).values
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))

Accuracy score : 0.9050987326849396


In [ ]:
predictions = model.predict(X_validation)

print("F1 score {0}".format(f1_score(predictions, Y_validation)))

F1 score 0.9472822527832351


In [ ]:
#So the raw data without any tranformations gives 0.910 accuracy and 0.95059 f1 score
#Let's transform the data by np.log because off high skewness

dataframe_tr = dataframe

In [ ]:
#For log scale the value can't be zero. So add 1 to every cell
dataframe_tr = dataframe_tr+1

#Convert to log scale
dataframe_tr = np.log(dataframe_tr)

In [ ]:
#Model fit and train
X = dataframe_tr.drop(columns=['label']).values
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

predictions = model.predict(X_validation)
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))
print("F1 score {0}".format(f1_score(predictions, Y_validation)))

Accuracy score : 0.9095195991747715
F1 score 0.9498284033338781


In [ ]:
#Lets deal with np.log and min max scaler

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(dataframe_tr.drop(columns=['label']).values)

#Transform and svae to X
X = scaler.transform(dataframe_tr.drop(columns=['label']).values)
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

predictions = model.predict(X_validation)
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))
print("F1 score {0}".format(f1_score(predictions, Y_validation)))

Accuracy score : 0.9115826702033598
F1 score 0.9509643674403401


In [ ]:
#Lets deal with np.log and Standard scaler

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(dataframe_tr.drop(columns=['label']).values)

#Transform and svae to X
X = scaler.transform(dataframe_tr.drop(columns=['label']).values)
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

predictions = model.predict(X_validation)
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))
print("F1 score {0}".format(f1_score(predictions, Y_validation)))

Accuracy score : 0.9106984969053935
F1 score 0.9503848043229081


In [ ]:
#Lets deal with np.log and Robust scaler

from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(dataframe_tr.drop(columns=['label']).values)

#Transform and svae to X
X = scaler.transform(dataframe_tr.drop(columns=['label']).values)
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

predictions = model.predict(X_validation)
print("Accuracy score : {0}".format(model.score(X_validation, Y_validation)))
print("F1 score {0}".format(f1_score(predictions, Y_validation)))

Accuracy score : 0.9106984969053935
F1 score 0.9504335023719941


In [ ]:
#Lets choose np.log and minmax and Grid search for parameters

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler

criterion = ['gini', 'entropy']
max_depth = [4,6,7,8,12,16,20,24,28]
splitter = ['best', 'random']
parameters = dict(criterion=criterion, max_depth=max_depth, splitter=splitter)
####Minmaxand log transformation start

scaler = MinMaxScaler()
scaler.fit(dataframe_tr.drop(columns=['label']).values)

#Transform and svae to X
X = scaler.transform(dataframe_tr.drop(columns=['label']).values)
Y = dataframe['label'].values
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.20, random_state=1)

####end of transformation

model = DecisionTreeClassifier()

clf = GridSearchCV(model, parameters)

# Fit the grid search
clf.fit(X_train, Y_train)

# View The Best Parameters
print('Best Criterion:', clf.best_estimator_.get_params()['criterion'])
print('Best max_depth:', clf.best_estimator_.get_params()['max_depth'])
print("Best score : ", clf.best_score_)

#Predictions
model = clf.best_estimator_
predictions = model.predict(X_validation)
print("F1 Score : ", f1_score(Y_validation, predictions))
clf.best_estimator_


Best Criterion: entropy
Best max_depth: 7
Best score :  0.9341193809874724
F1 Score :  0.965341812400636


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')